In [5]:
%load_ext autoreload
%autoreload 1

In [12]:
%aimport flange
import importlib
import networkx as nx

In [21]:
class inside(flange.FlangeTree):
    """Given a graph and a selector, returns a list of links that make up the min cut"""
    def __init__(self, selector):
        self.selector=selector

    def __call__(self, graph):
        selector = lambda x: self.selector(x, graph)
        nodes = list(filter(selector, graph.nodes())) ## TODO -- More complex query here
        synth = graph.subgraph(nodes) ## Get the subgraph indicated by the selector
        nx.set_edge_attributes(synth, 'capacity', 1)

        print("all nodes", nodes)
        print("init synth nodes:", synth.nodes())
        print("init synth edges:", synth.edges())
        outbound = list(graph.out_edges_iter(nodes))
        inbound = list(graph.in_edges_iter(nodes))
        print(inbound)
        print(outbound)

        src = "##SOURCE##"
        synth.add_node(src)
        for edge in inbound:
            if not synth.has_edge(*edge):
                synth.add_edge(src, edge[1], capacity=10)

        sink = "##SINK##"
        synth.add_node(sink)
        for edge in outbound:
            if not synth.has_edge(*edge):
                synth.add_edge(edge[0], sink, capacity=10)

        print("synth edges:", synth.edges())
        cut_value, partition = nx.minimum_cut(synth, s=src, t=sink, capacity='capacity')
        reachable, non_reachable = partition

        cutset = set()
        for u, nbrs in ((n, synth[n]) for n in reachable):
            cutset.update((u, v) for v in nbrs if v in non_reachable)
            cut_value == sum(synth.edge[u][v]['capacity'] for (u, v) in cutset)


        return cutset

In [22]:
flange = importlib.reload(flange)

In [23]:
i = inside(lambda x,g: int(g.node[x]["id"][-1]) < 2)(flange.graph("ring")())
r = {(e[0], e[1]) for e in i}
r

all nodes ['port1']
init synth nodes: ['port1']
init synth edges: []
[('port4', 'port1'), ('port2', 'port1')]
[('port1', 'port4'), ('port1', 'port2')]
synth edges: [('port1', '##SINK##'), ('##SOURCE##', 'port1')]


{('port1', '##SINK##')}

In [4]:
g = graph("ring")()

In [5]:
g.edges()

[('port1', 'port2'),
 ('port1', 'port4'),
 ('port2', 'port3'),
 ('port3', 'port4')]

In [4]:
dir(flange)

['ActionFailureError',
 'FlangeTree',
 'NoValidChoice',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'place',
 'update']